In [1]:
from character_mapping import Character_Map 
import theano 
import theano.tensor as T
import numpy as np 

text_test = './../texts/melville.txt'
char_map_obj = Character_Map(text_test,'mapping.dat',overwrite=True, break_line=None)
char_map_obj.k_map()
x_, y_, shared_x, shared_y = char_map_obj.gen_x_and_y(filename=None)

# print(shared_y.get_value().shape[1])
nh = 100
nx = len(char_map_obj.unique_char)
ny = nx 

Time creating character mapping and pickling: 1.9811 sec
Time creating k map 3.767 sec
Time creating arrays: 2.024 sec


<TensorType(float64, 3D)>
<TensorType(float64, 3D)>


In [64]:
import theano 
import theano.tensor as T 
import numpy as np 
from character_mapping import Character_Map
import time
try:
    reload
except NameError:
    from imp import reload
    
import RNN  
from RNN import RNNClass

reload(RNN)
from RNN import RNNClass

trainer = RNNClass(nh, nx, ny)
# trainer.train_no_index(training_data=(shared_x,shared_y),
#                        learning_rate=0.01,
#                        n_epochs=100,mini_batch_size=100)

# trainer.train_index(training_data=(shared_x,shared_y),
#                        learning_rate=0.01,
#                        n_epochs=100,mini_batch_size=100)

# x = T.matrix('x')
# y = T.matrix('y')


# cost = trainer.cross_entropy_loss(x,y)

# cost_fn = theano.function(
#     inputs = [x,y],
#     outputs = cost
#     )
# print("cost compiled")


function compiled


Time for minibatch: 0.279520988464
Sum for minibatch number 0 out of 793: 6.60274757715
Time for minibatch: 0.278197050095
Time for minibatch: 0.263979911804
Time for minibatch: 0.281971931458
Time for minibatch: 0.297733068466
Time for minibatch: 0.264256000519
Time for minibatch: 0.315468072891
Time for minibatch: 0.419100999832
Time for minibatch: 0.285853862762
Time for minibatch: 0.280938863754
Time for minibatch: 0.315122842789
Time for minibatch: 0.292364120483
Time for minibatch: 0.331890106201
Time for minibatch: 0.272921085358
Time for minibatch: 0.30301117897
Time for minibatch: 0.293340206146
Time for minibatch: 0.459197044373
Time for minibatch: 0.315034151077
Time for minibatch: 0.298177957535
Time for minibatch: 0.265707015991
Time for minibatch: 0.270560979843
Time for minibatch: 0.260865926743
Time for minibatch: 0.271103858948
Time for minibatch: 0.2741959095
Time for minibatch: 0.272019863129
Time for minibatch: 0.27101111412
Time for minibatch: 0

KeyboardInterrupt: 

In [70]:
import numpy as np
import time 
xs = T.tensor3('xs')
ys = T.tensor3('ys')
x0 = T.matrix('x0')

results1, updates1 = theano.scan(lambda prev_cost, xi, yi: trainer.cross_entropy_loss(xi, yi) + prev_cost,
                              sequences = [xs,ys],
                              outputs_info = trainer.cross_entropy_loss(xs[0],ys[0]))
results2, updates2 = theano.scan(lambda xi, yi: trainer.cross_entropy_loss(xi, yi),
                              sequences = [xs,ys])
                              #outputs_info = trainer.cross_entropy_loss(xs[0],ys[0]))
loss_fn1 = theano.function(
    inputs = [xs,ys],
    outputs = -T.mean(results1[-1])
)
loss_fn2 = theano.function(
    inputs = [xs,ys],
    outputs = -T.mean(results2)
)
print("loss_fn compiled")
t1 = time.time()
loss1 = loss_fn1(shared_x.get_value()[0:1000], shared_y.get_value()[0:1000])
print(time.time()-t1)
t2 = time.time()
loss2 = loss_fn2(shared_x.get_value()[0:1000], shared_y.get_value()[0:1000])
print(time.time()-t2)
# loss2 = -np.mean(loss1, axis=0)
# loss2.shape

loss_fn compiled
1.21108698845
1.30410695076
